In [5]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 7.4 MB/s eta 0:00:00


In [6]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# 1. Universe & Membership Functions
suhu = np.arange(0, 41, 1) # [cite: 202]
kelembapan = np.arange(0, 101, 1) # [cite: 203]
kipas = np.arange(0, 101, 1) # [cite: 204]

# Suhu
suhu_dingin = fuzz.trimf(suhu, [0, 0, 20]) # [cite: 208, 209]
suhu_hangat = fuzz.trimf(suhu, [15, 25, 35]) # [cite: 211, 212]
suhu_panas = fuzz.trimf(suhu, [30, 40, 40]) # [cite: 213]

# Kelembapan
hum_rendah = fuzz.trimf(kelembapan, [0, 0, 40]) # [cite: 216]
hum_sedang = fuzz.trimf(kelembapan, [30, 50, 70]) # [cite: 216]
hum_tinggi = fuzz.trimf(kelembapan, [60, 100, 100]) # [cite: 216]

# Kipas
kipas_pelan = fuzz.trimf(kipas, [0, 0, 50]) # [cite: 218]
kipas_sedang = fuzz.trimf(kipas, [25, 50, 75]) # [cite: 219]
kipas_cepat = fuzz.trimf(kipas, [50, 100, 100]) # [cite: 220]

In [7]:
def fuzzy_mamdani(suhu_val, kelembapan_val):
    # Fuzzifikasi
    mu_suhu_dingin = fuzz.interp_membership(suhu, suhu_dingin, suhu_val) # [cite: 227]
    mu_suhu_hangat = fuzz.interp_membership(suhu, suhu_hangat, suhu_val) # [cite: 227]
    mu_suhu_panas = fuzz.interp_membership(suhu, suhu_panas, suhu_val) # [cite: 228]

    mu_hum_rendah = fuzz.interp_membership(kelembapan, hum_rendah, kelembapan_val) # [cite: 229]
    mu_hum_sedang = fuzz.interp_membership(kelembapan, hum_sedang, kelembapan_val) # [cite: 229]
    mu_hum_tinggi = fuzz.interp_membership(kelembapan, hum_tinggi, kelembapan_val) # [cite: 229]

    # 3. Aturan Fuzzy (Mamdani)
    # Rule 1: IF Suhu dingin OR Kelembapan rendah THEN Kipas pelan
    alpha1 = np.fmax(mu_suhu_dingin, mu_hum_rendah) # [cite: 240]
    # Rule 2: IF Suhu hangat THEN Kipas sedang
    alpha2 = mu_suhu_hangat # [cite: 244]
    # Rule 3: IF Suhu panas OR Kelembapan tinggi THEN Kipas cepat
    alpha3 = np.fmax(mu_suhu_panas, mu_hum_tinggi) # [cite: 247]

    # 4. Implikasi (clipping)
    pelan_clipped = np.fmin(alpha1, kipas_pelan) # [cite: 262]
    sedang_clipped = np.fmin(alpha2, kipas_sedang) # [cite: 262]
    cepat_clipped = np.fmin(alpha3, kipas_cepat) # [cite: 262]

    # 5. Agregasi Output
    aggregated = np.fmax(pelan_clipped, np.fmax(sedang_clipped, cepat_clipped)) # [cite: 265]

    # 6. Defuzzifikasi
    centroid = fuzz.defuzz(kipas, aggregated, 'centroid') # [cite: 267]
    bisector = fuzz.defuzz(kipas, aggregated, 'bisector') # [cite: 268]
    mom = fuzz.defuzz(kipas, aggregated, 'mom') # [cite: 269]

    # 7. Visualisasi
    plt.figure(figsize=(8,4)) # [cite: 274]
    plt.plot(kipas, kipas_pelan, 'b', label='Pelan') # [cite: 276]
    plt.plot(kipas, kipas_sedang, 'g--', label='Sedang') # [cite: 279]
    plt.plot(kipas, kipas_cepat, 'r--', label='Cepat') # [cite: 281]
    plt.fill_between(kipas, aggregated, alpha=0.4, color='orange', label='Aggregated') # [cite: 284, 285]

    plt.axvline(centroid, color='k', linestyle='-', label=f'Centroid = {centroid:.1f}') # [cite: 289]
    plt.axvline(bisector, color='purple', linestyle=':', label=f'Bisector = {bisector:.1f}') # [cite: 289]
    plt.axvline(mom, color='brown', linestyle='--', label=f'MOM = {mom:.1f}') # [cite: 291]

    plt.title(f"Suhu={suhu_val:.1f}°C, Kelembapan={kelembapan_val:.1f}%") # [cite: 295]
    plt.xlabel("Kecepatan Kipas (%)") # [cite: 295]
    plt.ylabel("Derajat Keanggotaan") # [cite: 297]
    plt.ylim(-0.05, 1.05) # [cite: 299]
    plt.grid(True) # [cite: 301]
    plt.legend(loc='upper left', bbox_to_anchor=(1.01, 1)) # [cite: 304]
    plt.show() # [cite: 304]

    # Print hasil defuzzifikasi
    print(f"Defuzzifikasi:") # [cite: 309]
    print(f" Centroid = {centroid:.2f}%") # [cite: 311]
    print(f" Bisector = {bisector:.2f}%") # [cite: 313, 314]
    print(f" MOM      = {mom:.2f}%") # [cite: 316, 317]

In [8]:
# 8. Widget interaktif
interact(
    fuzzy_mamdani,
    suhu_val=FloatSlider(value=30, min=0, max=40, step=1, description='Suhu (°C)'), # [cite: 325]
    kelembapan_val=FloatSlider(value=70, min=0, max=100, step=1, description='Kelembapan (%)') # [cite: 326]
)

interactive(children=(FloatSlider(value=30.0, description='Suhu (°C)', max=40.0, step=1.0), FloatSlider(value=…

<function __main__.fuzzy_mamdani(suhu_val, kelembapan_val)>